In [39]:
!pip install google-generativeai
!pip install langchain
!pip install faiss-cpu
!pip install pandas


In [40]:
!pip install -U langchain-community

In [41]:
import os
import getpass
import pandas as pd
import google.generativeai as genai
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import GooglePalmEmbeddings
from langchain.chat_models import ChatGooglePalm
from langchain.memory import ConversationBufferMemory


In [42]:
from google.colab import userdata
API_KEY = userdata.get('Gemini')

os.environ["GOOGLE_API_KEY"] = API_KEY

genai.configure(api_key=API_KEY)


In [43]:
synthetic_data = {
    'content': [
        "Cashback Card offers 5% cashback on groceries.",
        "Travel Card offers 3x points on travel expenses.",
        "Economic outlook indicates a rise in inflation.",
        "Electronics purchases are increasing among young adults.",
        "Interest rates are expected to remain steady.",
        "Dining Card offers 10% cashback on dining expenses.",
        "Fashion Card offers 8% cashback on fashion purchases.",
        "GDP growth is projected to slow down next quarter.",
        "Unemployment rates are decreasing, indicating a strong job market.",
        "Fuel prices are expected to rise due to geopolitical tensions.",
    ]
}

df = pd.DataFrame(synthetic_data)

print("Synthetic Documents:")
print(df)


Synthetic Documents:
                                             content
0     Cashback Card offers 5% cashback on groceries.
1   Travel Card offers 3x points on travel expenses.
2    Economic outlook indicates a rise in inflation.
3  Electronics purchases are increasing among you...
4      Interest rates are expected to remain steady.
5  Dining Card offers 10% cashback on dining expe...
6  Fashion Card offers 8% cashback on fashion pur...
7  GDP growth is projected to slow down next quar...
8  Unemployment rates are decreasing, indicating ...
9  Fuel prices are expected to rise due to geopol...


In [44]:
df.to_csv('synthetic_data.csv', index=False)
print("Synthetic data saved to 'synthetic_data.csv'.")


Synthetic data saved to 'synthetic_data.csv'.


In [45]:
class UserProfile:
    def __init__(self, user_id, spending_patterns, card_type=None):
        self.user_id = user_id
        self.spending_patterns = spending_patterns  # e.g., {'groceries': 500, 'travel': 300}
        self.card_type = card_type  # The type of card the user currently holds
        self.card_offers = []
        self.economic_outlook = {}
        self.feedback = []

    def update_spending(self, category, amount):
        self.spending_patterns[category] = self.spending_patterns.get(category, 0) + amount

    def set_card_type(self, card_type):
        self.card_type = card_type

    def add_card_offer(self, offer):
        self.card_offers.append(offer)

    def set_economic_outlook(self, outlook):
        self.economic_outlook = outlook

    def add_feedback(self, feedback_text):
        self.feedback.append(feedback_text)


class UserContextManager:
    def __init__(self):
        self.user_profiles = {}

    def add_user_profile(self, user_profile):
        self.user_profiles[user_profile.user_id] = user_profile

    def get_user_profile(self, user_id):
        return self.user_profiles.get(user_id)

user_context_manager = UserContextManager()

In [46]:
class ContextMemoryManager:
    def __init__(self):
        self.context_memory = ConversationBufferMemory(
            memory_key="chat_history", input_key="question"
        )

    def get_context(self):
        return self.context_memory.load_memory_variables({})["chat_history"]

    def update_context(self, input_text, response_text):
        self.context_memory.save_context(
            {"question": input_text}, {"answer": response_text}
        )

context_memory_manager = ContextMemoryManager()


In [47]:
!pip install langchain_google_genai

In [48]:
class Card:
    def __init__(self, card_name, rewards_program, annual_fee, interest_rate, cashback_categories, card_type):
        """
        Initializes a card object.

        Parameters:
        - card_name: Name of the card.
        - rewards_program: Type of rewards (Cashback, Travel Miles, etc.).
        - annual_fee: Annual fee of the card.
        - interest_rate: Interest rate (APR) of the card.
        - cashback_categories: A dictionary that maps spending categories to cashback percentages.
          Example: {'Groceries': 5, 'Dining': 3}
        - card_type: The type of card (e.g., 'Premium', 'Standard', etc.).
        """
        self.card_name = card_name
        self.rewards_program = rewards_program
        self.annual_fee = annual_fee
        self.interest_rate = interest_rate
        self.cashback_categories = cashback_categories
        self.card_type = card_type

    def calculate_yearly_savings(self, user_spending):
        """
        Calculate the total yearly savings this card would provide for the user's spending patterns.

        Parameters:
        - user_spending: A dictionary containing the user's spending distribution across categories.

        Returns:
        - A dictionary with detailed breakdown of savings per category and total savings.
        """
        savings_details = {}
        total_savings = 0

        for category, amount in user_spending.items():
            if category in self.cashback_categories:
                cashback_rate = self.cashback_categories[category] / 100
                category_savings = amount * cashback_rate
                total_savings += category_savings
                savings_details[category] = category_savings

        net_savings = total_savings - self.annual_fee
        savings_details['total_savings'] = total_savings
        savings_details['net_savings'] = net_savings

        return savings_details

    def compare_with_user_card(self, user_card, user_spending):
        """
        Compare this card's savings with the user's current card.

        Parameters:
        - user_card: The user's current card (an instance of Card).
        - user_spending: A dictionary containing the user's spending distribution across categories.

        Returns:
        - A dictionary with the savings comparison.
        """
        current_card_savings = user_card.calculate_yearly_savings(user_spending)
        new_card_savings = self.calculate_yearly_savings(user_spending)

        comparison = {
            'current_card_savings': current_card_savings,
            'new_card_savings': new_card_savings,
            'savings_difference': new_card_savings['net_savings'] - current_card_savings['net_savings']
        }
        return comparison

    def __repr__(self):
        return f"{self.card_name} - {self.rewards_program} - Annual Fee: ${self.annual_fee:.2f} - Card Type: {self.card_type}"


In [49]:
def forecast_expenditure(user_spending, growth_rate=0.05, years=1):
    """
    Simulates future spending based on current spending and a growth rate.

    Parameters:
    - user_spending: A dictionary containing the user's current spending distribution across categories.
    - growth_rate: The projected yearly growth in spending (default is 5%).
    - years: The number of years to forecast spending (default is 1 year).

    Returns:
    - forecasted_spending: A dictionary with the projected future spending.
    """
    forecasted_spending = {}
    for category, amount in user_spending.items():
        forecasted_spending[category] = amount * ((1 + growth_rate) ** years)

    return forecasted_spending


In [50]:
import pandas as pd

transactions_df = pd.read_excel('transactions_with_card_types.xlsx')
cards_df = pd.read_excel('cards.xlsx')
economic_df = pd.read_excel('economic.xlsx')


transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])

user_spending_summary = transactions_df.groupby('UserID').agg(
    TotalSpending=('Amount', 'sum'),
    AvgTransactionAmount=('Amount', 'mean'),
    MostFrequentCategory=('Category', lambda x: x.mode()[0]),
    TransactionCount=('TransactionID', 'count')
).reset_index()

user_spending_summary = pd.merge(user_spending_summary, transactions_df[['UserID', 'CreditCardID','CardType']].drop_duplicates(), on='UserID', how='left')
user_spending_summary.rename(columns={'CardType_x':'CardType'},inplace=True)

cards_df['AnnualFee'] = cards_df['AnnualFee'].fillna(0)  # Replace missing fees with 0
cards_df['InterestRate'] = cards_df['InterestRate'].fillna(cards_df['InterestRate'].mean())  # Fill missing interest rates

# Merge user card data with the credit card inventory
user_spending_summary = pd.merge(user_spending_summary, cards_df, on='CreditCardID', how='left')

# Preprocess Economic Dataset
# Convert 'Date' to datetime
economic_df['Date'] = pd.to_datetime(economic_df['Date'])

# Get the latest economic data for the final date
latest_economic_data = economic_df[economic_df['Date'] == economic_df['Date'].max()].to_dict(orient='records')[0]

# Integrate economic conditions with user data
user_final_data = user_spending_summary.assign(**latest_economic_data)

print("User Data with Credit Card and Economic Data:")
print(user_final_data.head())


User Data with Credit Card and Economic Data:
      UserID  TotalSpending  AvgTransactionAmount MostFrequentCategory  \
0  UID000001        3266.81             54.446833             Clothing   
1  UID000001        3266.81             54.446833             Clothing   
2  UID000001        3266.81             54.446833             Clothing   
3  UID000001        3266.81             54.446833             Clothing   
4  UID000001        3266.81             54.446833             Clothing   

   TransactionCount CreditCardID   CardType_x  Unnamed: 0 CardType_y  \
0                60   CCID000691  Dining Card          35        NaN   
1                60   CCID000763  Dining Card          35        NaN   
2                60   CCID000870  Dining Card          35        NaN   
3                60   CCID000525  Dining Card          35        NaN   
4                60   CCID000637  Dining Card          35        NaN   

  CardLevel  ... RewardExpiry  SecurityFeatures  PaymentOptions IssuingBank 

In [51]:
class UserProfile:
    def __init__(self, user_id, spending_patterns, card_type):
        self.user_id = user_id
        self.spending_patterns = spending_patterns  # e.g., {'groceries': 500, 'travel': 300}
        self.card_type = card_type  # The type of card the user currently holds
        self.card_offers = []
        self.economic_outlook = {}
        self.feedback = []

    def update_spending(self, category, amount):
        self.spending_patterns[category] = self.spending_patterns.get(category, 0) + amount

    def set_card_type(self, card_type):
        self.card_type = card_type

    def add_card_offer(self, offer):
        self.card_offers.append(offer)

    def set_economic_outlook(self, outlook):
        self.economic_outlook = outlook

    def add_feedback(self, feedback_text):
        self.feedback.append(feedback_text)


class UserContextManager:
    def __init__(self):
        self.user_profiles = {}

    def add_user_profile(self, user_profile):
        self.user_profiles[user_profile.user_id] = user_profile

    def get_user_profile(self, user_id):
        return self.user_profiles.get(user_id)

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [53]:
genai.configure(api_key = API_KEY)

model = genai.GenerativeModel('gemini-pro')
df = pd.read_csv('synthetic_data.csv')

loader = DataFrameLoader(df, page_content_column='content')
documents = loader.load()

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
vector_store = FAISS.from_documents(documents, embeddings)



In [58]:
class Card:
    def __init__(self, card_name, rewards_program, annual_fee, interest_rate, cashback_categories, card_type):
        """
        Initializes a card object.

        Parameters:
        - card_name: Name of the card.
        - rewards_program: Type of rewards (Cashback, Travel Miles, etc.).
        - annual_fee: Annual fee of the card.
        - interest_rate: Interest rate (APR) of the card.
        - cashback_categories: A dictionary that maps spending categories to cashback percentages.
          Example: {'Groceries': 5, 'Dining': 3}
        - card_type: The type of card (e.g., 'Premium', 'Standard', etc.).
        """
        self.card_name = card_name
        self.rewards_program = rewards_program
        self.annual_fee = annual_fee
        self.interest_rate = interest_rate
        self.cashback_categories = cashback_categories
        self.card_type = self.card_name

    def calculate_yearly_savings(self, user_spending):
        """
        Calculate the total yearly savings this card would provide for the user's spending patterns.

        Parameters:
        - user_spending: A dictionary containing the user's spending distribution across categories.

        Returns:
        - A dictionary with detailed breakdown of savings per category and total savings.
        """
        savings_details = {}
        total_savings = 0

        for category, amount in user_spending.items():
            if category in self.cashback_categories:
                cashback_rate = self.cashback_categories[category] / 100
                category_savings = amount * cashback_rate
                total_savings += category_savings
                savings_details[category] = category_savings

        net_savings = total_savings - self.annual_fee
        savings_details['total_savings'] = total_savings
        savings_details['net_savings'] = net_savings

        return savings_details

    def compare_with_user_card(self, user_card, user_spending):
        """
        Compare this card's savings with the user's current card.

        Parameters:
        - user_card: The user's current card (an instance of Card).
        - user_spending: A dictionary containing the user's spending distribution across categories.

        Returns:
        - A dictionary with the savings comparison.
        """
        current_card_savings = user_card.calculate_yearly_savings(user_spending)
        new_card_savings = self.calculate_yearly_savings(user_spending)

        comparison = {
            'current_card_savings': current_card_savings,
            'new_card_savings': new_card_savings,
            'savings_difference': new_card_savings['net_savings'] - current_card_savings['net_savings']
        }
        return comparison

    def __repr__(self):
        return f"{self.card_name} - {self.rewards_program} - Annual Fee: ${self.annual_fee:.2f} - Card Type: {self.card_type}"

In [59]:
def suggest_best_cards(user_profile, cards):
    """
    Suggest the best cards based on the user's spending patterns with detailed numerical data.

    Parameters:
    - user_profile: UserProfile object containing the user's spending patterns.
    - cards: List of Card objects to evaluate.

    Returns:
    - A list of recommended cards with detailed savings analysis.
    """
    recommendations = []

    for card in cards:
        savings = card.calculate_yearly_savings(user_profile.spending_patterns)
        recommendations.append((card, savings))

    recommendations.sort(key=lambda x: x[1]['net_savings'], reverse=True)

    return recommendations[:3]


In [60]:
!pip install sentence_transformers

In [61]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Example of a lightweight model

categories = {
    'savings': "Tell me how much I can save with different cards.",
    'forecast': "Tell me the forecast of my spending for the next year.",
    'comparison': "Compare my current card with other options.",
    'general': "Give me general information about my financial status."
}

# Generate embeddings for each category description
category_embeddings = {category: model.encode(description) for category, description in categories.items()}

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [62]:
def classify_question_semantically(question):
    """
    Classifies the user's question based on semantic similarity using embeddings.

    Parameters:
    - question: The user's input question (string).

    Returns:
    - category: The category that best matches the question (savings, forecast, comparison, general).
    """
    question_embedding = model.encode(question)

    similarities = {category: util.pytorch_cos_sim(question_embedding, category_embedding) for category, category_embedding in category_embeddings.items()}

    best_match_category = max(similarities, key=similarities.get)

    return best_match_category


In [63]:
def generate_llm_prompt(user_profile, current_card, card_recommendations, forecasted_spending, question):
    """
    Builds a detailed prompt for the LLM based on the user's profile, question, and card data.
    The LLM will be responsible for crafting the response, including facts and numerical insights.

    Parameters:
    - user_profile: The user's profile containing spending patterns, card type, etc.
    - current_card: The user's current card.
    - card_recommendations: The best card recommendations for the user.
    - forecasted_spending: The user's projected future spending.
    - question: The user's specific question.

    Returns:
    - prompt: A formatted string containing all relevant details to pass to the LLM.
    """

    card_suggestions = ""
    for card, savings in card_recommendations:
        comparison = card.compare_with_user_card(current_card, user_profile.spending_patterns)
        card_suggestions += f"- {card.card_name}: Total Savings ${savings['total_savings']:.2f}, Net Savings after fee: ${savings['net_savings']:.2f}. "
        card_suggestions += f"You would save an additional ${comparison['savings_difference']:.2f} compared to your current card.\n"

    prompt = f"""
You are a financial assistant helping user {user_profile.user_id}. The user has asked the following question: "{question}"

User Profile:
- Spending Breakdown: {user_profile.spending_patterns}
- Current Card Type: {user_profile.card_type}
- Economic Outlook: Inflation rate is {user_profile.economic_outlook.get('inflation_rate')}%, Unemployment is {user_profile.economic_outlook.get('unemployment_rate')}%.

Expenditure Forecast (Next Year):
- Projected Spending: {forecasted_spending}

Card Recommendations:
{card_suggestions}

Please provide a detailed and helpful response based on this context. Include facts, comparisons, and suggestions that can assist the user with their question.
"""
    return prompt


In [64]:
def generate_custom_response(user_id, question):
    """
    Generates a custom response using the LLM, dynamically crafted based on user profile, spending data, and question.

    Parameters:
    - user_id: The ID of the user asking the question.
    - question: The specific question asked by the user.

    Returns:
    - A detailed and customized response.
    """
    user_profile = user_context_manager.get_user_profile(user_id)

    if not user_profile:
        return "User profile not found."

    current_card = None
    for card in cards:
        if card.card_type == user_profile.card_type:
            current_card = card
            break

    if not current_card:
        return f"Current card type '{user_profile.card_type}' not found in the system."

    card_recommendations = suggest_best_cards(user_profile, cards)

    forecasted_spending = forecast_expenditure(user_profile.spending_patterns, growth_rate=0.05, years=1)

    prompt = generate_llm_prompt(user_profile, current_card, card_recommendations, forecasted_spending, question)

    llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
    response = llm.predict(prompt)

    return response


In [65]:
def forecast_expenditure(user_spending, growth_rate=0.05, years=1):
    """
    Simulates future spending based on current spending and a growth rate.

    Parameters:
    - user_spending: A dictionary containing the user's current spending distribution across categories.
    - growth_rate: The projected yearly growth in spending (default is 5%).
    - years: The number of years to forecast spending (default is 1 year).

    Returns:
    - forecasted_spending: A dictionary with the projected future spending.
    """
    forecasted_spending = {}
    for category, amount in user_spending.items():
        forecasted_spending[category] = amount * ((1 + growth_rate) ** years)

    return forecasted_spending

In [66]:
user_context_manager = UserContextManager()

def create_users_from_data(user_data):
    for index, row in user_data.iterrows():
        spending_patterns = {row['MostFrequentCategory']: row['TotalSpending']}

        user_profile = UserProfile(row['UserID'], spending_patterns,card_type=row['CardType_x'])

        card_offer = f"{row['CardName']} with an interest rate of {row['InterestRate']}%"
        user_profile.add_card_offer(card_offer)

        economic_outlook = {
            'inflation_rate': row['InflationRate'],
            'unemployment_rate': row['UnemploymentRate'],
            'gdp': row['GDP']
        }
        user_profile.set_economic_outlook(economic_outlook)

        user_context_manager.add_user_profile(user_profile)

create_users_from_data(user_final_data)

print(f"Total Users Loaded: {len(user_context_manager.user_profiles)}")

# Define ContextMemoryManager class
class ContextMemoryManager:
    def __init__(self):
        self.context_memory = ConversationBufferMemory(
            memory_key="chat_history", input_key="question"
        )

    def get_context(self):
        return self.context_memory.load_memory_variables({})["chat_history"]

    def update_context(self, input_text, response_text):
        self.context_memory.save_context(
            {"question": input_text}, {"answer": response_text}
        )

context_memory_manager = ContextMemoryManager()

def generate_prompt(user_id, question):
    user_profile = user_context_manager.get_user_profile(user_id)

    if not user_profile:
        return "User profile not found."

    current_card = None
    for card in cards:
        if card.card_type == user_profile.card_type:
            current_card = card
            break


    card_recommendations = suggest_best_cards(user_profile, cards)

    forecasted_spending = forecast_expenditure(user_profile.spending_patterns, growth_rate=0.05, years=1)

    card_suggestions = ""
    for card, savings in card_recommendations:
        comparison = card.compare_with_user_card(current_card, user_profile.spending_patterns)
        card_suggestions += f"\n- {card.card_name} (Comparison with current {current_card.card_name}):"
        for category, amount_saved in savings.items():
            if category not in ['total_savings', 'net_savings']:
                card_suggestions += f"\n  - Savings from {category}: ${amount_saved:.2f}"
        card_suggestions += f"\n  - Total Savings: ${savings['total_savings']:.2f}"
        card_suggestions += f"\n  - Net Savings (after annual fee): ${savings['net_savings']:.2f}"
        card_suggestions += f"\n  - Compared to your current card, you would save an additional ${comparison['savings_difference']:.2f}\n"

    prompt = f"""
You are a financial assistant helping user {user_profile.user_id}.Dont give template answers answer paying attention to{question} and give asnwers as a person. Give very detailed and numerically backed answers,crunch as much data as you can.


User Profile:
- Total Spending Breakdown: {user_profile.spending_patterns}
- Current Card Type: {user_profile.card_type}
- Economic Outlook: Inflation rate is {user_profile.economic_outlook.get('inflation_rate')}%, Unemployment is {user_profile.economic_outlook.get('unemployment_rate')}%.

Expenditure Forecast (Next Year):
- Projected Spending: {forecasted_spending}

Based on your spending habits, we recommend the following cards with detailed savings analysis:
{card_suggestions}

Please provide detailed advice based on the above, and answer the following question:
"{question}"
"""
    return prompt


df = pd.read_csv('synthetic_data.csv')

loader = DataFrameLoader(df, page_content_column='content')
documents = loader.load()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(documents, embeddings)

def retrieve_documents(query):
    similar_docs = vector_store.similarity_search(query, k=3)
    return '\n'.join([doc.page_content for doc in similar_docs])

def generate_response(prompt):
    llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
    response = llm.predict(prompt)
    return response

def validate_response(response, retrieved_info):
    for fact in retrieved_info.split('\n'):
        if fact.strip() and fact.strip() not in response:
            return False
    return True

def collect_feedback(user_id, feedback_text):
    user_profile = user_context_manager.get_user_profile(user_id)
    if user_profile:
        user_profile.add_feedback(feedback_text)

Total Users Loaded: 1000


In [67]:
cards = [
    Card("Cashback Groceries Card", "Cashback", 99, 15.99, {'Groceries': 5, 'Dining': 1},'Economic'),
    Card("Travel Rewards Card", "Travel Miles", 150, 18.99, {'Travel': 3, 'Dining': 2},'Standard'),
    Card("Dining Card", "Cashback", 50, 12.99, {'Dining': 4, 'Groceries': 2},'Standard'),
    Card("Fashion Rewards Card", "Cashback", 75, 14.99, {'Fashion': 3, 'Travel': 1},'Premium'),
    Card("General Rewards Card", "Points", 0, 16.99, {'Groceries': 1, 'Travel': 1, 'Dining': 1},'Premium'),
]

In [68]:
def chat():
    user_id = input("Enter your user ID (e.g., 'UID000001'): ")

    if user_context_manager.get_user_profile(user_id) is None:
        print("User ID not found. Please try again.")
        return

    print("Type 'exit' or 'quit' to end the chat.")

    while True:
        question = input("\nYou: ")
        if question.lower() in ['exit', 'quit']:
            print("Chatbot: Thank you for using the service. Goodbye!")
            break

        response = generate_custom_response(user_id, question)


        retrieved_info = retrieve_documents(question)
        is_valid = validate_response(response, retrieved_info)


        print(f"\nChatbot:\n{response}")

        context_memory_manager.update_context(question, response)

        feedback = input("\nWas this response helpful? (yes/no): ")
        if feedback.lower() == 'no':
            feedback_text = input("Please provide your feedback to help us improve: ")
            collect_feedback(user_id, feedback_text)
            print("Thank you for your feedback!")
        else:
            print("Glad to hear that!")


In [69]:
chat()

Enter your user ID (e.g., 'UID000001'): UID000001
Type 'exit' or 'quit' to end the chat.

You:  based on my previous year credit card expenses how do you suggest to save more

Chatbot:
**Based on your previous year's credit card expenses, here are some suggestions to help you save more:**

**1. Track your expenses:** Use a budgeting app or spreadsheet to track your spending. This will help you identify areas where you can cut back.

**2. Set a budget:** Once you know where your money is going, you can create a budget to help you stay on track. Make sure to include both fixed expenses (such as rent or mortgage) and variable expenses (such as groceries or entertainment).

**3. Negotiate lower interest rates:** If you have credit card debt, call your creditors and see if you can negotiate a lower interest rate. This can save you a significant amount of money over time.

**4. Use a balance transfer credit card:** If you have multiple credit cards with high interest rates, consider consolid